<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

--2023-06-09 01:36:51--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.009s  

2023-06-09 01:36:51 (267 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
!unzip -q spa-eng.zip

In [3]:
text_file = "spa-eng/spa.txt"

In [22]:
with open(text_file) as f:
  lines = f.read().split("\n")[:-1] 

In [5]:
text_pair = []

In [24]:
with open("spa-eng/san.txt", 'w') as w:
  w.writelines("Santosh sarangi \n123 780 \nHello \nZebra animal")

In [28]:
with open("spa-eng/san.txt") as f:
  lines = f.read().split("\n")

In [29]:
lines

['Santosh sarangi ', '123 780 ', 'Hello ', 'Zebra animal']